# Метрики качества классификации

## 1. Данные

In [1]:
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns

In [2]:
from sklearn.model_selection import train_test_split

In [3]:
X = pd.read_pickle('data/shelter/res/xtrain.pkl')
y = pd.read_csv('data/shelter/train.csv')['OutcomeType']

для учебных примеров оставим только 2 класса, отличающихся значительно и по смыслу, и по размеру

In [4]:
y.value_counts()

Adoption           10769
Transfer            9422
Return_to_owner     4786
Euthanasia          1555
Died                 197
Name: OutcomeType, dtype: int64

In [5]:
is_first_2_class = (y=='Adoption')|(y=='Euthanasia')

In [6]:
X = X[is_first_2_class]
X.index = range(len(X))
y = y[is_first_2_class]
y.index = range(len(y))

y теперь можно перевести в множество {0,1}

In [7]:
y = (y=='Adoption').astype(int)

для оценки качества разобьём выборки на 2 части, 3:1

In [8]:
Xtr, Xval, ytr, yval = train_test_split(X, y, test_size=0.25)

-----

## 2. Обучим классификатор

In [9]:
from sklearn.tree import DecisionTreeClassifier

In [10]:
clf = DecisionTreeClassifier(max_depth=5)

In [11]:
clf.fit(Xtr, ytr)

DecisionTreeClassifier(max_depth=5)

In [12]:
yval_pred = clf.predict(Xval)
yval_pred_proba = clf.predict_proba(Xval)

-----

## 3. Написание метрик с нуля

In [13]:
from sklearn import metrics

**hint:** в сами библиотеки можно заходить и смотрить код. Особенно это удобно в [PyCharm](https://www.jetbrains.com/pycharm/)

-----

Основа многих метрик - матрица ошибок (confusion matrix)

**Задание.** Написать функцию для расчёта матрицы ошибок при заданных *y, y_pred* и сравнить со значениями из sklearn.metrics

In [32]:
def get_confusion_matrix(y, y_pred, print_matrix=False):
    # приводим это в pandas.Series и избавляемся от индекса 
    # (иначе по нему будет идти сопоставление)
    y = pd.Series(list(y))
    y_pred = pd.Series(list(y_pred))
    TP=0
    FP=0
    FN=0
    TN=0
    for i in range (0, len(y)):

        #Вы предсказали положительное, и это правда
        if (y[i]==1)and(y_pred[i]==1):
            TP=1+TP

        #Вы предсказали положительное значение, и это неверно
        elif (y[i]==0)and(y_pred[i]==1):
            FP=1+FP
        
        #Вы предсказали отрицательное значение, и это неверно
        elif (y[i]==1)and(y_pred[i]==0):
            FN=1+FN
        
        #Вы прогнозировали отрицательное значения, и это правда
        elif (y[i]==0)and(y_pred[i]==0):
            TN=1+TN
    
    # выводим матрицу
    if print_matrix:
        print("{:5} {:5} {:5}".format("", "y=1", "y=0"))
        print("a=1 {:5} {:5}".format(TP, FP))
        print("a=0 {:5} {:5}".format(FN, TN))
    
    # возврашает функция словарь результатов
    return {"TP":TP, "FP":FP, "TN":TN, "FN":FN}

In [33]:
get_confusion_matrix(yval, yval_pred, True)

      y=1   y=0  
a=1  2661   157
a=0    22   241


{'TP': 2661, 'FP': 157, 'TN': 241, 'FN': 22}

Сравним полученные значения с встроеной функцией

In [34]:
print(metrics.confusion_matrix(yval, yval_pred))

[[ 241  157]
 [  22 2661]]


-----

**Задание.** Написать функцию для расчёта accuracy (доли верных ответов) и сравнить со значением из sklearn.metrics

$accuracy = \frac {TP+TN}{TP+TN+FP+FN}$

2901 3080

In [46]:
def accuracy(y, y_pred):
    # для простоты передадим ранее написаную функцию МАТРИЦЫ ОШИБОК и из нее возьмем результаты
    matrix=get_confusion_matrix(y, y_pred, print_matrix=False)
    TP=matrix['TP']
    FP=matrix['FP']
    TN=matrix['TN']
    FN=matrix['FN']   

    accuracy=(TP+TN)/(TP+TN+FP+FN) 
    return accuracy

    

Сравниваем со значением библиотечным

In [47]:
accuracy(yval, yval_pred)

0.9419019798766635

In [48]:
print(metrics.accuracy_score(yval, yval_pred))

0.9419019798766635


-----

**Задание.** Написать функцию для расчёта precision (точности) и сравнить со значением из sklearn.metrics

$precision = \frac {TP}{TP+FP}$

In [49]:
def precision(y, y_pred):
        # для простоты передадим ранее написаную функцию МАТРИЦЫ ОШИБОК и из нее возьмем результаты
    matrix=get_confusion_matrix(y, y_pred, print_matrix=False)
    
    TP=matrix['TP']
    FP=matrix['FP']
    precision=(TP)/(TP+FP)
    return precision

In [50]:
precision(yval, yval_pred)

0.9442867281760113

In [51]:
print(metrics.precision_score(yval, yval_pred))

0.9442867281760113


-----

**Задание.** Написать функцию для расчёта recall (полноты) и сравнить со значением из sklearn.metrics

$recall = \frac {TP}{TP+FN}$

In [52]:
def recall(y, y_pred):
    # для простоты передадим ранее написаную функцию МАТРИЦЫ ОШИБОК и из нее возьмем результаты
    matrix=get_confusion_matrix(y, y_pred, print_matrix=False)
    
    TP=matrix['TP']
    FN=matrix['FN']  

    recall=TP/(TP+FN)
    return recall

In [53]:
recall(yval, yval_pred)

0.9918002236302647

In [54]:
print(metrics.recall_score(yval, yval_pred))

0.9918002236302647


-----

**Задание.** Написать функцию для расчёта f1 score и сравнить со значением из sklearn.metrics

$f1 = 2*\frac {precision*recall}{precision+recall}$

In [57]:
def f1_score(y, y_pred):
    re=recall(y, y_pred)
    pc=precision(y, y_pred)

    f1=2*(pc*re)/(pc+re)
    return f1

In [58]:
f1_score(yval, yval_pred)

0.9674604617342302

In [59]:
print(metrics.f1_score(yval, yval_pred))

0.9674604617342302


-----

**Задание.** Найти значения AUC-ROC, AUR-PRC, logloss с помощью sklearn

In [60]:
metrics.roc_auc_score(yval, yval_pred_proba[:,1])

0.9234796794258284

In [61]:
metrics.average_precision_score(yval, yval_pred_proba[:,1])

0.982205833504191

In [62]:
metrics.log_loss(yval, yval_pred_proba[:,1])

0.19742300940925658